## Uvod

Ova lekcija će obuhvatiti:  
- Što je pozivanje funkcije i njene primjene  
- Kako stvoriti poziv funkcije koristeći OpenAI  
- Kako integrirati poziv funkcije u aplikaciju  

## Ciljevi učenja

Nakon završetka ove lekcije znat ćete kako i razumjeti:  

- Svrhu korištenja pozivanja funkcije  
- Postavljanje poziva funkcije koristeći OpenAI servis  
- Dizajnirati učinkovite pozive funkcija za slučaj upotrebe vaše aplikacije  


## Razumijevanje poziva funkcija

Za ovu lekciju želimo izgraditi značajku za naš startup za obrazovanje koja korisnicima omogućuje korištenje chatbota za pronalaženje tehničkih tečajeva. Preporučivat ćemo tečajeve koji odgovaraju njihovoj razini vještina, trenutnoj ulozi i tehnologiji od interesa.

Za dovršetak koristit ćemo kombinaciju:
 - `OpenAI` za stvaranje chat iskustva za korisnika
 - `Microsoft Learn Catalog API` za pomoć korisnicima u pronalaženju tečajeva na temelju zahtjeva korisnika
 - `Function Calling` za preuzimanje upita korisnika i slanje funkciji za izvršavanje API zahtjeva.

Za početak, pogledajmo zašto bismo uopće željeli koristiti pozive funkcija:

print("Poruke u sljedećem zahtjevu:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dobivanje novog odgovora od GPT-a gdje može vidjeti odgovor funkcije


print(second_response.choices[0].message)


### Zašto pozivanje funkcija

Ako ste završili bilo koju drugu lekciju u ovom tečaju, vjerojatno razumijete moć korištenja velikih jezičnih modela (LLM-ova). Nadamo se da također možete vidjeti neke njihove ograničenosti.

Pozivanje funkcija je značajka OpenAI servisa dizajnirana za rješavanje sljedećih izazova:

Nekonzistentno formatiranje odgovora:
- Prije pozivanja funkcija, odgovori velikog jezičnog modela bili su nestrukturirani i nekonzistentni. Programeri su morali pisati složen kod za validaciju kako bi obradili svaku varijaciju u izlazu.

Ograničena integracija s vanjskim podacima:
- Prije ove značajke, bilo je teško uključiti podatke iz drugih dijelova aplikacije u kontekst razgovora.

Standardiziranjem formata odgovora i omogućavanjem besprijekorne integracije s vanjskim podacima, pozivanje funkcija pojednostavljuje razvoj i smanjuje potrebu za dodatnom logikom validacije.

Korisnici nisu mogli dobiti odgovore poput "Kakvo je trenutno vrijeme u Stockholmu?". To je zato što su modeli bili ograničeni na vrijeme na koje su podaci trenirani.

Pogledajmo primjer u nastavku koji ilustrira ovaj problem:

Recimo da želimo stvoriti bazu podataka o studentima kako bismo im mogli predložiti pravi tečaj. Ispod imamo dva opisa studenata koja su vrlo slična u podacima koje sadrže.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Želimo ovo poslati LLM-u da analizira podatke. Ovo se kasnije može koristiti u našoj aplikaciji za slanje na API ili pohranu u bazu podataka.

Napravimo dva identična upita u kojima ćemo LLM-u objasniti koje informacije nas zanimaju:


Želimo ovo poslati LLM-u da analizira dijelove koji su važni za naš proizvod. Tako možemo stvoriti dva identična upita za uputiti LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Nakon što stvorimo ove dvije upute, poslat ćemo ih LLM-u koristeći `openai.ChatCompletion`. Uputu spremamo u varijablu `messages` i dodjeljujemo joj ulogu `user`. Ovo je kako bismo oponašali poruku koju korisnik piše chatbotu.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Sada možemo poslati oba zahtjeva LLM-u i ispitati odgovor koji primimo.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Iako su upiti isti, a opisi slični, možemo dobiti različite formate svojstva `Grades`.

Ako pokrenete gornju ćeliju više puta, format može biti `3.7` ili `3.7 GPA`.

To je zato što LLM uzima nestrukturirane podatke u obliku napisanog upita i također vraća nestrukturirane podatke. Potrebno je imati strukturirani format kako bismo znali što očekivati prilikom pohrane ili korištenja tih podataka.

Korištenjem funkcijskog poziva možemo osigurati da dobijemo strukturirane podatke natrag. Prilikom korištenja funkcijskog poziva, LLM zapravo ne poziva niti izvršava bilo koje funkcije. Umjesto toga, stvaramo strukturu koju LLM treba slijediti u svojim odgovorima. Zatim koristimo te strukturirane odgovore da bismo znali koju funkciju pokrenuti u našim aplikacijama.


![Dijagram toka poziva funkcije](../../../../translated_images/hr/Function-Flow.083875364af4f4bb.webp)


Zatim možemo uzeti ono što funkcija vrati i poslati to natrag LLM-u. LLM će tada odgovoriti koristeći prirodni jezik kako bi odgovorio na korisnički upit.


### Primjeri upotrebe poziva funkcija

**Pozivanje vanjskih alata**  
Chatbotovi su izvrsni u pružanju odgovora na pitanja korisnika. Korištenjem poziva funkcija, chatbotovi mogu koristiti poruke korisnika za izvršavanje određenih zadataka. Na primjer, student može zatražiti od chatbota da "Pošalje e-mail mom predavaču s porukom da mi treba dodatna pomoć oko ovog predmeta". To može pokrenuti poziv funkcije `send_email(to: string, body: string)`

**Izrada API ili baza podataka upita**  
Korisnici mogu pronaći informacije koristeći prirodni jezik koji se zatim pretvara u formatirani upit ili API zahtjev. Primjer za to može biti nastavnik koji pita "Tko su studenti koji su završili zadnji zadatak" što može pozvati funkciju nazvanu `get_completed(student_name: string, assignment: int, current_status: string)`

**Izrada strukturiranih podataka**  
Korisnici mogu uzeti blok teksta ili CSV i koristiti LLM za izdvajanje važnih informacija iz njega. Na primjer, student može pretvoriti Wikipedia članak o mirovnim sporazumima u AI flash kartice. To se može napraviti korištenjem funkcije nazvane `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Kreiranje vašeg prvog poziva funkcije

Proces kreiranja poziva funkcije uključuje 3 glavna koraka:
1. Pozivanje Chat Completions API-ja s popisom vaših funkcija i korisničkom porukom
2. Čitanje odgovora modela kako biste izvršili radnju, tj. izvršili funkciju ili API poziv
3. Izvršavanje još jednog poziva Chat Completions API-ja s odgovorom vaše funkcije kako biste koristili te informacije za kreiranje odgovora korisniku.


![Tijek poziva funkcije](../../../../translated_images/hr/LLM-Flow.3285ed8caf4796d7.webp)


### Elementi poziva funkcije

#### Unos korisnika

Prvi korak je kreirati korisničku poruku. To se može dinamički dodijeliti uzimanjem vrijednosti iz tekstualnog unosa ili možete ovdje dodijeliti vrijednost. Ako je ovo vaš prvi put da radite s Chat Completions API-jem, trebamo definirati `role` i `content` poruke.

`role` može biti `system` (kreiranje pravila), `assistant` (model) ili `user` (krajnji korisnik). Za pozivanje funkcije, dodijelit ćemo ovo kao `user` i primjer pitanja.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Kreiranje funkcija.

Sljedeće ćemo definirati funkciju i parametre te funkcije. Ovdje ćemo koristiti samo jednu funkciju nazvanu `search_courses`, ali možete kreirati više funkcija.

**Važno**: Funkcije su uključene u sistemsku poruku za LLM i bit će uključene u količinu dostupnih tokena koje imate na raspolaganju.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicije** 

Struktura definicije funkcije ima više razina, svaka sa svojim svojstvima. Evo pregleda ugniježđene strukture:

**Svojstva funkcije na najvišoj razini:**

`name` - Ime funkcije koju želimo pozvati. 

`description` - Ovo je opis kako funkcija radi. Ovdje je važno biti specifičan i jasan. 

`parameters` - Popis vrijednosti i formata koje želite da model proizvede u svom odgovoru. 

**Svojstva objekta parametara:**

`type` - Tip podataka objekta parametara (obično "object")

`properties` - Popis specifičnih vrijednosti koje će model koristiti za svoj odgovor. 

**Svojstva pojedinačnih parametara:**

`name` - Implicitno definirano ključem svojstva (npr. "role", "product", "level")

`type` - Tip podataka ovog specifičnog parametra (npr. "string", "number", "boolean") 

`description` - Opis specifičnog parametra. 

**Opcionalna svojstva:**

`required` - Niz koji navodi koji su parametri obavezni da bi se poziv funkcije izvršio.


### Pozivanje funkcije  
Nakon definiranja funkcije, sada je potrebno uključiti je u poziv Chat Completion API-ja. To radimo dodavanjem `functions` u zahtjev. U ovom slučaju `functions=functions`.  

Postoji i opcija postaviti `function_call` na `auto`. To znači da ćemo dopustiti LLM-u da odluči koja funkcija treba biti pozvana na temelju korisničke poruke, umjesto da to sami dodijelimo.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sada pogledajmo odgovor i vidimo kako je formatiran:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Možete vidjeti da je ime funkcije pozvano i iz korisničke poruke, LLM je uspio pronaći podatke koji odgovaraju argumentima funkcije.


## 3. Integracija poziva funkcija u aplikaciju.

Nakon što smo testirali formatirani odgovor iz LLM-a, sada to možemo integrirati u aplikaciju.

### Upravljanje tijekom

Da bismo to integrirali u našu aplikaciju, poduzmimo sljedeće korake:

Prvo, napravimo poziv OpenAI uslugama i spremimo poruku u varijablu nazvanu `response_message`.


In [ ]:
response_message = response.choices[0].message

Sada ćemo definirati funkciju koja će pozvati Microsoft Learn API za dobivanje popisa tečajeva:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kao najbolja praksa, zatim ćemo provjeriti želi li model pozvati funkciju. Nakon toga, stvorit ćemo jednu od dostupnih funkcija i uskladiti je s funkcijom koja se poziva.  
Zatim ćemo uzeti argumente funkcije i preslikati ih na argumente iz LLM-a.

Na kraju, dodati ćemo poruku poziva funkcije i vrijednosti koje je vratila poruka `search_courses`. To daje LLM-u sve informacije koje su mu potrebne za  
odgovor korisniku koristeći prirodni jezik.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Sada ćemo poslati ažuriranu poruku LLM-u kako bismo mogli primiti odgovor u prirodnom jeziku umjesto odgovora u JSON formatu API-ja.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Izazov s kodom

Odličan posao! Za nastavak učenja o OpenAI Function Callingu možete izgraditi: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Više parametara funkcije koji bi mogli pomoći učenicima da pronađu više tečajeva. Dostupne API parametre možete pronaći ovdje:  
 - Kreirajte još jedan poziv funkcije koji uzima više informacija od učenika, poput njihovog materinjeg jezika  
 - Kreirajte rukovanje pogreškama kada poziv funkcije i/ili API poziv ne vrati odgovarajuće tečajeve  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Odricanje od odgovornosti**:
Ovaj dokument preveden je pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo postići točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za kritične informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
